In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
#inputs for clay loam
phi_ae=630 #mm
b=8.52
kh=0.000245 #cm/s
kh=kh*10*60*60 #mm/hr
n=0.476
theta_i=np.array([0.18,0.24,0.30,0.36,0.42]) #inital volumetric soil moisture content
catchment_frac=np.array([0.40,0.25,0.20,0.10,0.05])
# inputs for watershed
w= 30 #mm/h
dur=4 #hr

In [62]:
kp

NameError: name 'kp' is not defined

In [68]:
ks=kh*(2/3)
phi_f=phi_ae*((2*b+3)/(2*b+6))
sp=np.sqrt(kh*(n-theta_i)*phi_f)
tc=sp*sp/(4*(w-ks)*(w-ks))
tp=(sp*sp*(2*w-ks))/(4*w*(w-ks)*(w-ks))
ts=tp-tc

In [69]:
ks

5.88

In [70]:
tc

array([0.61475418, 0.49014185, 0.36552951, 0.24091718, 0.11630484])

In [71]:
tp

array([1.10901654, 0.88421589, 0.65941524, 0.43461459, 0.20981394])

In [72]:
time_step=np.concatenate([np.array([0.01]),np.arange(0.025,0.076,0.025),np.arange(0.1,24.1,0.1)])

In [73]:
np.shape(time_step)

(244,)

In [74]:
df1=df2=df3=df4=df5=pd.DataFrame(time_step,columns=['time'])

In [75]:
sp

array([37.82318039, 33.77288724, 29.16543931, 23.67779102, 16.45152652])

In [76]:
tp[0]

1.109016539271924

In [77]:
df1['f']=""

In [78]:
df1.head()

,time,f
0,0.010,
1,0.025,
2,0.050,
3,0.075,
4,0.100,


In [79]:
#https://stackoverflow.com/questions/50932651/perform-a-conditional-operation-on-a-pandas-column
#df.loc[df['W'] < 1, 'W'] = df.loc[df['W'] < 1, 'W'] * 2
mask=df1.time<tp[0]
tef=tp-tc
df1.loc[df1['time']<tp[0], 'f']=w
df1.loc[df1['time']>tp[0],'f']=(df1.loc[df1['time']>tp[0]]['time'])-tef[0]
df1.loc[df1['time']>tp[0],'f']=sp[0]*np.power(df1.loc[df1['time']>tp[0],'f'],-0.5)+ks

In [80]:
tef[0]

0.49426236007462687

In [81]:
df1.head(10)

,time,f
0,0.010,30
1,0.025,30
2,0.050,30
3,0.075,30
4,0.100,30
5,0.200,30
6,0.300,30
7,0.400,30
8,0.500,30
9,0.600,30


In [82]:
df1['F']=""
df1.loc[df1['time']<tp[0],'F']=df1.loc[df1['time']<tp[0],'f']*df1.loc[df1['time']<tp[0],'time']

In [83]:
df1.loc[df1['time']>tp[0],'F']=df1.loc[df1['time']>tp[0],'f']*(df1.loc[df1['time']>tp[0],'time']-tef[0])

In [84]:
df1.head(10)

,time,f,F
0,0.010,30,0.3
1,0.025,30,0.75
2,0.050,30,1.5
3,0.075,30,2.25
4,0.100,30,3
5,0.200,30,6
6,0.300,30,9
7,0.400,30,12
8,0.500,30,15
9,0.600,30,18


In [85]:
tc

array([0.61475418, 0.49014185, 0.36552951, 0.24091718, 0.11630484])

In [86]:
tp

array([1.10901654, 0.88421589, 0.65941524, 0.43461459, 0.20981394])